<a href="https://colab.research.google.com/github/Sarthakpunj/CV_analyser/blob/main/Final_CV_ANALYZER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pdfplumber
!pip install python-docx
import pandas as pd
import numpy as np
from sklearn import linear_model
import pdfplumber
from docx import Document
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import Counter
import re
from io import BytesIO

# Define the Model Class
class train_model:
    def train(self):
        data = pd.read_csv('training_dataset.csv')
        array = data.values

        for i in range(len(array)):
            if array[i][0] == "Male":
                array[i][0] = 1
            else:
                array[i][0] = 0

        df = pd.DataFrame(array)
        maindf = df[[0, 1, 2, 3, 4, 5, 6]]
        mainarray = maindf.values
        temp = df[7]
        train_y = temp.values

        self.mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg', max_iter=1000)
        self.mul_lr.fit(mainarray, train_y)

    def test(self, test_data):
        try:
            test_predict = [int(i) for i in test_data]
            y_pred = self.mul_lr.predict([test_predict])
            return y_pred[0]
        except:
            return "All factors for finding personality not entered!"

# Define Text Extraction Functions
def extract_text_from_pdf(file_content):
    with pdfplumber.open(BytesIO(file_content)) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

def extract_text_from_docx(file_content):
    doc = Document(BytesIO(file_content))
    text = ''
    for para in doc.paragraphs:
        text += para.text + '\n'
    return text

# ... (Continue defining analysis, prediction, and GUI functions)
def extract_text_from_docx(file_content):
    doc = Document(BytesIO(file_content))
    text = ''
    for para in doc.paragraphs:
        text += para.text + '\n'
    return text

# Define Analysis Functions
def analyze_resume(text, model, desired_skill):
    """Analyze the resume text to provide strengths and improvement areas."""
    keywords = ['experience', 'education', 'certifications', 'achievements',
                'leadership', 'teamwork', 'communication', 'problem-solving', 'project management']
    # Sample in-demand skills (replace with a more comprehensive list)
    in_demand_skills = ['python', 'machine learning', 'data analysis', 'communication', 'project management']
    analysis = {'strengths': [], 'weaknesses': [], 'improvements': [], 'matched_skills': []}

    # Convert text to lower case and split into words
    text_lower = text.lower()
    words = re.findall(r'\w+', text_lower)

    # Keyword Frequency Analysis
    word_freq = Counter(words)
    for keyword in keywords:
        if keyword in word_freq:
            analysis['strengths'].append(f"Strong emphasis on '{keyword}'")
        else:
            analysis['improvements'].append(f"Consider adding more about '{keyword}'")

    # Identify Experience Level
    experience_years = re.findall(r'\d+\s*(?:years?|yrs?)', text_lower)
    if experience_years:
        total_experience = sum(int(re.findall(r'\d+', exp)[0]) for exp in experience_years)
        analysis['strengths'].append(f"Estimated total experience: {total_experience} years")

    # Skill Matching
    for skill in in_demand_skills:
        if skill in text_lower:
            analysis['matched_skills'].append(skill)

    # Identify Potential Weaknesses (Example)
    if 'teamwork' not in text_lower and 'collaboration' not in text_lower:
        analysis['weaknesses'].append("Lack of explicit mention of teamwork or collaboration experience.")

    return analysis

# Define the Prediction Result Function
def prediction_result(file_content, file_extension, model, desired_skill):
    clear_output()

    # Extract text from resume content based on file extension
    if file_extension == '.pdf':
        resume_text = extract_text_from_pdf(file_content)
    elif file_extension == '.docx':
        resume_text = extract_text_from_docx(file_content)
    else:
        resume_text = "Unsupported file type"

    # Analyze resume for strengths and improvements
    analysis = analyze_resume(resume_text, model, desired_skill)
    print("\n############# Resume Analysis #############\n")

    print("Strengths:")
    for strength in analysis['strengths']:
        print(f"- {strength}")

    if analysis['weaknesses']:
        print("\nPotential Weaknesses:")
        for weakness in analysis['weaknesses']:
            print(f"- {weakness}")

    print("\nAreas for Improvement:")
    for improvement in analysis['improvements']:
        print(f"- {improvement}")

    # Skill Matching
    if desired_skill in resume_text.lower():
        print(f"\nThe skill '{desired_skill}' was found in the resume.")
    else:
        print(f"\nThe skill '{desired_skill}' was not found in the resume.")

# Define the GUI Function
def run_gui():
    clear_output()
    uploader = widgets.FileUpload(
        accept='.pdf,.docx',
        multiple=False
    )

    skill_input = widgets.Text(
        description='Enter Skill:',
        placeholder='e.g., Python, Data Analysis'
    )

    submit_button = widgets.Button(description="Submit")

    def on_submit(button):
        if uploader.value:
            file_info = list(uploader.value.values())[0]
            file_content = file_info['content']
            file_extension = file_info['metadata']['name'].split('.')[-1]
            file_extension = '.' + file_extension
            desired_skill = skill_input.value.lower() # Get skill input from user
            prediction_result(file_content, file_extension, model, desired_skill) # Pass desired skill to prediction_result

    submit_button.on_click(on_submit)

    display(uploader, skill_input, submit_button)

# Initialize and Run the Model
model = train_model()
model.train()
run_gui()



############# Resume Analysis #############

Strengths:
- Strong emphasis on 'achievements'

Potential Weaknesses:
- Lack of explicit mention of teamwork or collaboration experience.

Areas for Improvement:
- Consider adding more about 'experience'
- Consider adding more about 'education'
- Consider adding more about 'certifications'
- Consider adding more about 'leadership'
- Consider adding more about 'teamwork'
- Consider adding more about 'communication'
- Consider adding more about 'problem-solving'
- Consider adding more about 'project management'

The skill 'a' was found in the resume.
